In [1]:
# Data
import numpy as np

with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [2]:
# Model or Network
import impl.layer as l

class GRU:
    def __init__(self, D, H, L, char2idx, idx2char):
        self.D = D
        self.H = H
        self.L = L
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        self.losses = {'train':[], 'smooth train':[]}
        
        # Model params
        Z = H + D
        m = dict(
            Wz=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wr=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wh=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(H / 2.),
            bz=np.zeros((1, H)),
            br=np.zeros((1, H)),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

        self.model = []
        for layer in range(self.L):
            self.model.append(m)

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, m):
        Wz, Wr, Wh, Wy = m['Wz'], m['Wr'], m['Wh'], m['Wy']
        bz, br, bh, by = m['bz'], m['br'], m['bh'], m['by']

        X_one_hot = X.copy()
        h_old = h.copy()

        X = np.column_stack((h_old, X_one_hot))

        hz, hz_cache = l.fc_forward(X, Wz, bz)
        hz, hz_sigm_cache = l.sigmoid_forward(hz)

        hr, hr_cache = l.fc_forward(X, Wr, br)
        hr, hr_sigm_cache = l.sigmoid_forward(hr)

        X_prime = np.column_stack((hr * h_old, X_one_hot))
        hh, hh_cache = l.fc_forward(X_prime, Wh, bh)
        hh, hh_tanh_cache = l.tanh_forward(hh)

        h = (1. - hz) * h_old + hz * hh

        y, y_cache = l.fc_forward(h, Wy, by)

        cache = (
            X, X_prime, h_old, hz, hz_cache, hz_sigm_cache, hr, hr_cache, hr_sigm_cache,
            hh, hh_cache, hh_tanh_cache, h, y_cache
        )

        return y, h, cache

    def backward(self, dy, dh, cache):
        X, X_prime, h_old, hz, hz_cache, hz_sigm_cache, hr, hr_cache, hr_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache = cache
        
        dh_next = dh.copy()

        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next

        dhh = hz * dh
        dh_old1 = (1. - hz) * dh
        dhz = hh * dh - h_old * dh

        dhh = l.tanh_backward(dhh, hh_tanh_cache)
        dX_prime, dWh, dbh = l.fc_backward(dhh, hh_cache)

        dh_prime = dX_prime[:, :self.H]
        dh_old2 = hr * dh_prime

        dhr = h_old * dh_prime
        dhr = l.sigmoid_backward(dhr, hr_sigm_cache)
        dXr, dWr, dbr = l.fc_backward(dhr, hr_cache)

        dhz = l.sigmoid_backward(dhz, hz_sigm_cache)
        dXz, dWz, dbz = l.fc_backward(dhz, hz_cache)

        dX = dXr + dXz
        dh_old3 = dX[:, :self.H]

        dh = dh_old1 + dh_old2 + dh_old3
        dX = dX[:, self.H:]

        grad = dict(Wz=dWz, Wr=dWr, Wh=dWh, Wy=dWy, bz=dbz, br=dbr, bh=dbh, by=dby)
        
        return dX, dh, grad

    def train_forward(self, X_train, h):
        ys, caches = [], []
        h_init = h.copy()
        h = []
        for _ in range(self.L):
            h.append(h_init.copy())
            caches.append([])

        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            y = X_one_hot.reshape(1, -1)
            for layer in range(self.L):
                y, h[layer], cache = self.forward(y, h[layer], self.model[layer])
                caches[layer].append(cache)
            ys.append(y)
            
        return ys, caches

    def cross_entropy(self, y_pred, y_train):
        m = y_pred.shape[0]

        prob = l.softmax(y_pred)
        log_like = -np.log(prob[range(m), y_train])
        data_loss = np.sum(log_like) / m

        return data_loss

    def dcross_entropy(self, y_pred, y_train):
        m = y_pred.shape[0]

        grad_y = l.softmax(y_pred)
        grad_y[range(m), y_train] -= 1.0
        grad_y /= m

        return grad_y
    
    def loss_function(self, y_train, ys):
        loss, dys = 0.0, []

        for y_pred, y in zip(ys, y_train):
            loss += self.cross_entropy(y_pred, y)
            dy = self.dcross_entropy(y_pred, y)
            dys.append(dy)
            
        return loss, dys
    
    def train_backward(self, dys, caches):
        dh, grad, grads = [], [], []
        for _ in range(self.L):
            dh.append(np.zeros((1, self.H)))
            grad.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
            grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
        
        for t in reversed(range(len(dys))):
            dX = dys[t]
            for layer in reversed(range(self.L)):
                dX, dh[layer], grad[layer] = self.backward(dX, dh[layer], caches[layer][t])
                for k in grad[0].keys():
                    grads[layer][k] += grad[layer][k]
                
        return dX, grads
    
    def test(self, X_seed, h, size):
        chars = [self.idx2char[X_seed]]
        idx_list = list(range(self.vocab_size))
        X = X_seed
        
        h_init = h.copy()
        h = []
        for _ in range(self.L):
            h.append(h_init.copy())

        for _ in range(size):
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            y = X_one_hot.reshape(1, -1)
            for layer in range(self.L):
                y, h[layer], _ = self.forward(y, h[layer], self.model[layer])
                
            prob = l.softmax(y)
            idx = np.random.choice(idx_list, p=prob.ravel())
            chars.append(self.idx2char[idx])
            X = idx

        return ''.join(chars)

In [3]:
def get_minibatch(X, y, minibatch_size, shuffle):
    minibatches = []

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]
        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha, mb_size, n_iter, print_after):

    M, R = [], []
    for layer in range(nn.L):
        M.append({k: np.zeros_like(v) for k, v in nn.model[layer].items()})
        R.append({k: np.zeros_like(v) for k, v in nn.model[layer].items()})
        
    beta1 = .99
    beta2 = .999
    idx = 0
    state = nn.initial_state()
    smooth_loss = 1 # -np.log(1.0 / len(set(X_train)))
    eps = 1e-8
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)
    
    for iter in range(1, n_iter + 1):
        for idx in range(len(minibatches)):
            X_mini, y_mini = minibatches[idx]
            ys, caches = nn.train_forward(X_mini, state)
            loss, dys = nn.loss_function(y_mini, ys)
            _, grads = nn.train_backward(dys, caches)
            nn.losses['train'].append(loss)
            smooth_loss = (0.999 * smooth_loss) + (0.001 * loss)
            nn.losses['smooth train'].append(smooth_loss)

            for layer in range(nn.L):
                for k in grads[layer].keys(): #key, value: items
                    M[layer][k] = l.exp_running_avg(M[layer][k], grads[layer][k], beta1)
                    R[layer][k] = l.exp_running_avg(R[layer][k], grads[layer][k]**2, beta2)

                    m_k_hat = M[layer][k] / (1. - (beta1**(iter)))
                    r_k_hat = R[layer][k] / (1. - (beta2**(iter)))

                    nn.model[layer][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + eps)

        # Print loss and test sample
        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, loss))
            sample = nn.test(X_mini[0], state, 100)
            print(sample)

    return nn

In [ ]:
# Hyper-parameters
time_step = 100 # width, minibatch size and test sample size as well
num_layers = 1 # depth
n_iter = 10000 # epochs
alpha = 1e-4 # learning_rate
p_dropout = 0.95 # q=1-p, q=keep_prob and p=dropout.
print_after = 10 # n_iter//10 # print training loss, valid, and test
num_hidden_units = 64 # num_hidden_units in hidden layer
num_input_units = len(char_to_idx) # vocab_size = len(char_to_idx)

# Build the network and learning it or optimizing it using SGD
net = GRU(D=num_input_units, H=num_hidden_units, L=num_layers, char2idx=char_to_idx, idx2char=idx_to_char)

# Start learning using BP-SGD-ADAM
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, print_after=print_after)

# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(net.losses['train'], label='Train loss')
plt.plot(net.losses['smooth train'], label='Train smooth loss')
plt.legend()
plt.show()

Iter-10 loss: 104.4002
e-Retciis,re  s  th ehe nfee stp aaar innuho,n  ea fat5olshaDnoohL  u eineueino med. ptleuay9y in y l
Iter-20 loss: 102.6579
e'row undmren oi dha1e tfmelaf oB f R e Ne, p hia7 mgnnmrrrdsiMous ake  i toouiles Ua momp-nrinninyia
Iter-30 loss: 100.3455
ea1G.i  -pltunite z as y rtpe upsarde fantauipa ls, uy as b-dtos usss 1htehoarno'0cln7igemeceviNe rf 
Iter-40 loss: 98.3492
e0Bk JnJ,on tipeh nit ta2 4afe eioaios oJh  heir  rwhuFntdEy,psn,gJmket7  thxIpono ry lan  ssoa ai al
Iter-50 loss: 96.6431
eaihs uyst-tedaldWpvrinas uhEomeattokina he uumtrkin tb2ir at aninm ten my snl rin teJags eine a tin 
Iter-60 loss: 95.1867
e6-M日gtd7 arat iotfeinat vte'rs  karsh5drhisa.ithe(l8 an orkd ntfs tgitrec heoTlpretsa  Jo tar WTscf 
Iter-70 loss: 93.9353
e0O5r teTen r7o15s phe9bJJWerpatis Aa anneouisiIcegras fgItxru ttretigwiO emid kauapteaRss.  ansn ire
Iter-80 loss: 92.8418
emabs "odJm ale cnn9 roWan 0hoic thc rpCoar mash oAd if iee iNathes mm ms rieis lig sire aheitoT fe a
Iter-

Iter-670 loss: 54.9543
eWd eunthe Emperia lollestu font lycunses fors aEdestur of Japangi. Japaianas paritalofelthy %il texs
Iter-680 loss: 54.2634
evtres1. Rp:B teloD. Japan ia o2 Hlama er and by Chinau, wam illor 9, 12s arlicigans lediticasl Lonld
Iter-690 loss: 53.5825
eiwing in thech wod this, Tomithernly dpper ho mithe inthin mated Seani hflh'  somha ast an expan oth
Iter-700 loss: 52.9091
ef Japan se poruth. ConWurges fhif the coas the the tao Sea ir China, mtoningtitu id tad 17t on Nibct
Iter-710 loss: 52.2407
en;. Japapen is the Emperioen, untpis ittary of The Waslo-Japan's mangeccigest pvint to thu Sta is th
Iter-720 loss: 51.5753
e0tlano latas indexinn Serlo's tho Ease worstoreand is contir of Japan, the Muvio , manden puringead 
Iter-730 loss: 50.9117
et intho, Anof of Ocelast in hud eaman's atudy. Arucearel Purcodent pereate, ind a9 Eare O3 of1", ank
Iter-740 loss: 50.5543
ejo Japan Sut ricem. The Guriaa, Japandes tikoredidi, 1nE1this wat p9egea a  Emperea at ty clkofome f


Iter-1330 loss: 33.0888
e enoun cefoiy sivy cound-Japanise Was talurye Ities. Arthisea fid thec. N2itanoge the "Land of chict
Iter-1340 loss: 32.8965
ed the worldCs anded in of the Gstof ctulivase cenhopente somn Aeininy Cropec, wax fourthc ted lepuro
Iter-1350 loss: 32.6807
er abd ligs,mthd I, the Wat, OParangent enor Nolaturs a lits o, the tis dhan is rontorias paryes. The
Iter-1360 loss: 32.4678
esind-itatirs, ADia ly hich-cand of in inther omlate Wistond begsente perters dhd sisth westire the E
Iter-1370 loss: 32.3458
egestaten of the sobledide is thn wourto kobelcoand. Japanesi, whe wixm ntokoalecy mian in 186, Btant
Iter-1380 loss: 32.2305
eulaty in lary slamak". Japan so ar ho hist tpendevy Rused.5% Setho skao poped atirn. China ly murte 
Iter-1390 loss: 31.9001
ex and te meelins o mpio ts it deth c perla stalyvist ctbiobal-oralico ante world's tingd Sia ctalyy 
Iter-1400 loss: 31.7333
en wictrenio fed impcition"s war ermaldy worst rons ef owed in itulevo liges of 1bound. Japan 

Iter-1990 loss: 22.4367
eConn ethinast rentues. China Sea, chty pes unced in the nomlice isotr hisidad foust. Japan popel wai
Iter-2000 loss: 22.5462
eg conterin an eso Japan") isstict riparuliChan slomalcengunstiof Japan", was erthic memliky bom ecla
Iter-2010 loss: 22.3193
eAsinad fourt. Japan") is an is, and ivplliror, Japan surkha as porgh ty preclang, the the G20. Japan
Iter-2020 loss: 22.2062
e Nithe peniopro, chastrived umintan Gl018, ind 1, the Upper ardes. Chind Worth Japan 6, Wirt. R Asta
Iter-2030 loss: 22.0514
ef the East Indexint rechanes ano-nominalico in, whicha, the woth Nvinn"Sunded inceand gentuccounire 
Iter-2040 loss: 21.8875
ed an is aansy fedurombea of fourtad icmanes upati a d the cati of Toky. The nopleth s :isored tic ta
Iter-2050 loss: 22.5478
es and rounto Toellaints perie tho Fst 19.5 Couna-dyfins a dilted pinge Seato Japanese, insuctertitit
Iter-2060 loss: 21.8051
en corlareld fous the fitp oxpllarym, whic pares, ahd country Tokuly in Cound lion mevinnsk ao

Iter-2650 loss: 15.4911
ean has Hurancea, nhectuoys wed u7 ha e-dLyanowes. The country is the Meiji Empioto chaler Ooolacin",
Iter-2660 loss: 15.3702
e markemiis larg sinsti the ha eso the SesGome the ares at the G20 and isare gingd fourth and cankem 
Iter-2670 loss: 15.2545
e bemfricte pertoeno, narghases in thichue-te aso the worrd's in oppan es Wont Stotion marledexsas ma
Iter-2680 loss: 15.1189
esty upto ains. Japan is a dethe The Global tor cealykyid negstala"gy an ex and toust Aprenkec o mfly
Iter-2690 loss: 14.9252
e conf ucthongg sited a menteonomf of expande Wor highm iom tte opon untee the Upper -popmpfres lives
Iter-2700 loss: 15.0782
e coubalicogeclamiel Wwothe Nevhoust repioto cyy in io levh ctandec iogt with rog ou toe chind-sofmGb
Iter-2710 loss: 15.0498
en, the anse hound ig toss. Chin" I ante paries, rnied a unthe ,. Japan's lame of an is the Furst rit
Iter-2720 loss: 15.0124
e tored parisit riginogSun"L wobontoflic danges liclde ind rese coum ef tyo he as th mandest p

...

Iter-6610 loss: 3.4925
es ander. Japan has chatilatar endin withopes in tue world. The Grurtal ommpinstu ar histd re tory Ch
Iter-6620 loss: 3.4658
e makleusingwa tid redest rigite in the Glo, ritodecea ,edge porto-Japane op whi Up,e h rmbirol Cokna
Iter-6630 loss: 3.4246
er and Winter Olympic Games.
. Tie nine es aty tod ropalcom. fiontand seated rened"pande,curev of Hip
Iter-6640 loss: 3.3928
e world's largest meirowits a dever comniof ingline Worla, Japan") iwa , Sand of as it the sourtlly. 
Iter-6650 loss: 4.0680
ed narnedins an of laict inte and, 685she Conndef fimplobs lall war emclristuc lome bomsinh aso to tr
Iter-6660 loss: 3.7590
egenta lytitomenla is the sixto lomitaonncy coningha as entediveneng ntred,- architatity it the 12th 
Iter-6670 loss: 3.6650
ed larkes, world's regomrevsinane gimmed it to he tosed ay in the Sestid China is, chinapics and to a
Iter-6680 loss: 3.5830
ed nhichuits re expod the Rising und fivere ming population. The tour erghisted andera ding anse aint


Iter-7270 loss: 2.8652
en the nasb Chinh surchasan, warter and aane gomndectiry s r19to and Rusing, Japan, the Ease ward Cop
Iter-7280 loss: 2.9075
ed first in the number of Nobel laureates of any country in Asia. Japan is ranked first in the Countr
Iter-7290 loss: 2.9214
ed first in the number of Nobel laureates of any country in Asia. Japan is ranked first in the Countr
Iter-7300 loss: 2.9075
e toryl Consted fers bediot it tobs th gheshacest lile irstarny Brand ucoranc. anounfinssived fion ri
Iter-7310 loss: 2.9100
e world's largest metropolitan area with over 35 million residents and the world's largest wrourllico
Iter-7320 loss: 2.8856
er and Winter Olympic Games.
. Tie nine es lolitso dicimenker irge G8, an militiry it orghs lered 20u
Iter-7330 loss: 2.9143
ed rankec frco, which inctu seagandenasina in wethirol corld toud thk sexEas  orghina wirtet of encon
Iter-7340 loss: 2.8862
es aiterunm lfous Japanesane mollecbinfes in the caul rity o 47 frest cakn op peat-kae ina Snce Gfeut


Iter-7930 loss: 2.3739
ed first in the numbor of Toe the tupro lide surghagestan"Lamakec, white io the Meijid Chinasico and 
Iter-7940 loss: 2.3594
e norlly bomletitinntu ae sturtoko. Okkat ov rol of es arded in the Empore caly the "s ato opoly. in 
Iter-7950 loss: 2.3438
e world's largest metropolitan area an eront in in the worl Chicanica and chikaloko lecrulita of Japa
Iter-7960 loss: 2.5951
ed narnegilster Astorngina worerth  orflowed filsturom, the 17th centurionted inted inalirg ot the 1s
Iter-7970 loss: 2.5256
ed first in the number of Japan, whice is lirg les en op boliona Inaex and the Risina, Japan wose Pac
Iter-7980 loss: 2.5112
ery Brand Indecine sudth partest. Approfids hes. Necrntmiod intan Astal comkercl mefrrestod Stat of t
Iter-7990 loss: 2.4166
er and Winter Olympic Games.
. Tai suese with ith the Ripenge gorlt. The fourtly forstryt in the coul
Iter-8000 loss: 2.3671
er and Winter Olympic Games.
. Tie pins o paritalo lomuich ince wargist tioun mfioto and in the world


Iter-8590 loss: 2.2552
e world's largest metropolitan area with over 35 million residelits itoma ic aigstokio fid to emperto
Iter-8600 loss: 2.2431
empio, the highest miel on iorlatid andexpan es ancethe ercodic ai sexto-y. Frease frllironion iry ci
Iter-8610 loss: 2.2314
ered indichandef fiurt-koid regitstre China Sto lyt tis. The pippperlokI fers ald tiind Japan esil "s
Iter-8620 loss: 2.2201
ean world hag solaty of Okel wor durce torrd lane Sul livistolion in tuese Glomaideinnimp lolanceaing
Iter-8630 loss: 2.2094
er and Winter Olympic Games.
.
. Tie pankes Wic livina and countey in 1868, whiten ndentient mitres f
Iter-8640 loss: 2.1990
e world's largest metropolitan area Wint io ald the trerthe the curlakion in toest Sedio kicuter ommn
Iter-8650 loss: 2.1887
ered into a long period of isolation in the early 17th century, which was ended in 1853 parntaines, a
Iter-8660 loss: 2.1784
e nrofiry it exporthy Brand Indexwar eEd Sna es frest. Inge andect of iwal Dieth nankea in the G201at


Iter-9250 loss: 2.1327
ed first in the number of Nitelnte perian, whith sham siontykies in the esc the sixth ing fu17th Nper
Iter-9260 loss: 2.0756
elorederint. The forulalicy an erolowity s stveropan intred ingtin the Gos larl Chinanisonany conlamb
Iter-9270 loss: 2.0630
e world's largest metropolitan area with over 35 million residents and the world's largest urban aggl
Iter-9280 loss: 2.0617
e norllyity ingtuna, whe thirn"sunany 9La, the worldsith-rapneghiniginl curion-lire Sea or the Oigllo
Iter-9290 loss: 2.0595
er and Winter Olympic Games.
. Tie ninese Chind o lation in the OECD and the fourth lountucary of Oet
Iter-9300 loss: 2.0601
ered into a long period of isolation in the early 17th century, which was ended in 1853 when a United
Iter-9310 loss: 2.0609
er and Wend ra lodids pwrirectio  intinglimiflis mailadena and an expllioanfics cfletionte pente nope
Iter-9320 loss: 2.0598
economy by shecerid as a lima ectndecsuryv per e pandecais sed first-koined nysinlad it anoso Japanes
